In [1]:
import pandas as pd
import numpy as np
import os

Reference: https://dhiraj-p-rai.medium.com/logistic-regression-in-spark-ml-8a95b5f5434c

In [2]:
import pyspark
from pyspark.sql.functions import col
from pyspark.sql.functions import when

In [3]:
spark = pyspark.sql.SparkSession.builder.master('local[1]').appName('H1B').getOrCreate()
test_data = spark.read.csv('train.csv', header = True, inferSchema = True)

21/10/31 00:49:44 WARN Utils: Your hostname, Renas-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 172.20.17.36 instead (on interface en0)
21/10/31 00:49:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/10/31 00:49:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
21/10/31 00:49:46 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
test_data = spark.read.csv('test.csv', header = True, inferSchema = True)

In [6]:
cols = training_data.columns
cols.remove('EMPLOYER_NAME')

In [7]:
training_data = training_data.drop(col('EMPLOYER_NAME'))
training_data = training_data.withColumn("CASE_DURATION",col("CASE_DURATION").cast("int"))
training_data = training_data.withColumn("FULL_TIME_POSITION",col("FULL_TIME_POSITION").cast("int"))
test_data = test_data.drop(col('EMPLOYER_NAME'))
test_data = test_data.withColumn(
    "CASE_DURATION", col("CASE_DURATION").cast("int"))
test_data = test_data.withColumn(
    "FULL_TIME_POSITION", col("FULL_TIME_POSITION").cast("int"))


In [8]:
from pyspark.ml.feature import VectorAssembler
try:
  cols.remove('CASE_STATUS')
except:
  pass
cols.remove('_c0')
training_data = training_data.drop(col('_c0'))
test_data = test_data.drop(col('_c0'))
assembler = VectorAssembler(inputCols=cols,outputCol="features", handleInvalid='skip')


In [9]:
training_data=assembler.transform(training_data)
test_data = assembler.transform(test_data)


In [ ]:
training_data.select("features").show()

In [10]:
training_data = training_data.withColumn("CASE_STATUS",
                                         when(training_data.CASE_STATUS == 'CERTIFIED',1)
                                         .otherwise(when(training_data.CASE_STATUS == 'DENIED',0)
                                         .otherwise(when(training_data.CASE_STATUS == 'WITHDRAWN',3)
                                         .otherwise(2))))
test_data = test_data.withColumn("CASE_STATUS",
                                         when(test_data.CASE_STATUS ==
                                              'CERTIFIED', 1)
                                         .otherwise(when(test_data.CASE_STATUS == 'DENIED', 0)
                                                    .otherwise(when(test_data.CASE_STATUS == 'WITHDRAWN', 3)
                                                    .otherwise(2))))


In [11]:
from pyspark.ml.classification import LogisticRegression

In [12]:
lr = LogisticRegression(labelCol = 'CASE_STATUS',
                        featuresCol = 'features',
                        maxIter=10, regParam=0, elasticNetParam=0.8).setFamily("multinomial")

In [15]:
model = lr.fit(training_data)

21/10/31 00:59:45 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: AK, AL, AR, AZ, CA, CO, CT, DC, DE, FL, FM, GA, GU, HI, IA, ID, IL, IN, KS, KY, LA, MA, MD, ME, MH, MI, MN, MO, MP, MS, MT, NC, ND, NE, NH, NJ, NM, NV, NY, OH, OK, OR, PA, PR, PW, RI, SC, SD, TN, TX, UT, VA, VI, VT, WA, WI, WV, WY, 11-1, 11-2, 11-3, 11-9, 13-1, 13-2, 15-1, 15-2, 15-9, 16-1, 17-1, 17-2, 17-3, 19-1, 19-2, 19-3, 19-4, 20-2, 21-1, 21-2, 23-1, 23-2, 25-1, 25-2, 25-3, 25-4, 25-7, 25-9, 26-1, 27-1, 27-2, 27-3, 27-4, 29-1, 29-2, 29-9, 31-1, 31-2, 31-9, 33-1, 33-2, 33-3, 33-9, 35-1, 35-2, 35-3, 35-9, 37-1, 37-2, 37-3, 39-1, 39-2, 39-3, 39-4, 39-5, 39-6, 39-7, 39-9, 40-9, 41-1, 41-2, 41-3, 41-4, 41-9, 43-1, 43-3, 43-4, 43-5, 43-6, 43-9, 45-1, 45-2, 45-3, 45-4, 47-1, 47-2, 47-3, 47-4, 47-5, 49-1, 49-2, 49-3, 49-9, 51-1, 51-2, 51-3, 51-4, 51-5, 51-6, 51-7, 51-8, 51-9, 53-1, 53-2, 53-3, 53-5, 53-6, 53-7, 54-1, 71-2, E-3 Australian, H-1B, H-1B1 Chile, H-1B1 Singapore, AK, AL, AR, AS, AZ, CA,

In [16]:
predict_train = model.transform(training_data)
predict_test = model.transform(test_data)


In [37]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol='CASE_STATUS', probabilityCol = 'probability', metricName='f1')
predict_test.printSchema()


root
 |-- AK1: integer (nullable = true)
 |-- AL2: integer (nullable = true)
 |-- AR3: integer (nullable = true)
 |-- AZ4: integer (nullable = true)
 |-- CA5: integer (nullable = true)
 |-- CO6: integer (nullable = true)
 |-- CT7: integer (nullable = true)
 |-- DC8: integer (nullable = true)
 |-- DE9: integer (nullable = true)
 |-- FL10: integer (nullable = true)
 |-- FM11: integer (nullable = true)
 |-- GA12: integer (nullable = true)
 |-- GU13: integer (nullable = true)
 |-- HI14: integer (nullable = true)
 |-- IA15: integer (nullable = true)
 |-- ID16: integer (nullable = true)
 |-- IL17: integer (nullable = true)
 |-- IN18: integer (nullable = true)
 |-- KS19: integer (nullable = true)
 |-- KY20: integer (nullable = true)
 |-- LA21: integer (nullable = true)
 |-- MA22: integer (nullable = true)
 |-- MD23: integer (nullable = true)
 |-- ME24: integer (nullable = true)
 |-- MH: integer (nullable = true)
 |-- MI26: integer (nullable = true)
 |-- MN27: integer (nullable = true)
 |-- MO

In [38]:
evaluator.evaluate(predict_test)

21/10/31 01:18:14 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: AK, AL, AR, AZ, CA, CO, CT, DC, DE, FL, FM, GA, GU, HI, IA, ID, IL, IN, KS, KY, LA, MA, MD, ME, MH, MI, MN, MO, MP, MS, MT, NC, ND, NE, NH, NJ, NM, NV, NY, OH, OK, OR, PA, PR, PW, RI, SC, SD, TN, TX, UT, VA, VI, VT, WA, WI, WV, WY, 11-1, 11-2, 11-3, 11-9, 13-1, 13-2, 15-1, 15-2, 15-9, 16-1, 17-1, 17-2, 17-3, 19-1, 19-2, 19-3, 19-4, 20-2, 21-1, 21-2, 23-1, 23-2, 25-1, 25-2, 25-3, 25-4, 25-7, 25-9, 26-1, 27-1, 27-2, 27-3, 27-4, 29-1, 29-2, 29-9, 31-1, 31-2, 31-9, 33-1, 33-2, 33-3, 33-9, 35-1, 35-2, 35-3, 35-9, 37-1, 37-2, 37-3, 39-1, 39-2, 39-3, 39-4, 39-5, 39-6, 39-7, 39-9, 40-9, 41-1, 41-2, 41-3, 41-4, 41-9, 43-1, 43-3, 43-4, 43-5, 43-6, 43-9, 45-1, 45-2, 45-3, 45-4, 47-1, 47-2, 47-3, 47-4, 47-5, 49-1, 49-2, 49-3, 49-9, 51-1, 51-2, 51-3, 51-4, 51-5, 51-6, 51-7, 51-8, 51-9, 53-1, 53-2, 53-3, 53-5, 53-6, 53-7, 54-1, 71-2, E-3 Australian, H-1B, H-1B1 Chile, H-1B1 Singapore, AK, AL, AR, AS, AZ, CA,

0.926217960159995

In [39]:
evaluator.evaluate(predict_train)


21/10/31 01:18:51 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: AK, AL, AR, AZ, CA, CO, CT, DC, DE, FL, FM, GA, GU, HI, IA, ID, IL, IN, KS, KY, LA, MA, MD, ME, MH, MI, MN, MO, MP, MS, MT, NC, ND, NE, NH, NJ, NM, NV, NY, OH, OK, OR, PA, PR, PW, RI, SC, SD, TN, TX, UT, VA, VI, VT, WA, WI, WV, WY, 11-1, 11-2, 11-3, 11-9, 13-1, 13-2, 15-1, 15-2, 15-9, 16-1, 17-1, 17-2, 17-3, 19-1, 19-2, 19-3, 19-4, 20-2, 21-1, 21-2, 23-1, 23-2, 25-1, 25-2, 25-3, 25-4, 25-7, 25-9, 26-1, 27-1, 27-2, 27-3, 27-4, 29-1, 29-2, 29-9, 31-1, 31-2, 31-9, 33-1, 33-2, 33-3, 33-9, 35-1, 35-2, 35-3, 35-9, 37-1, 37-2, 37-3, 39-1, 39-2, 39-3, 39-4, 39-5, 39-6, 39-7, 39-9, 40-9, 41-1, 41-2, 41-3, 41-4, 41-9, 43-1, 43-3, 43-4, 43-5, 43-6, 43-9, 45-1, 45-2, 45-3, 45-4, 47-1, 47-2, 47-3, 47-4, 47-5, 49-1, 49-2, 49-3, 49-9, 51-1, 51-2, 51-3, 51-4, 51-5, 51-6, 51-7, 51-8, 51-9, 53-1, 53-2, 53-3, 53-5, 53-6, 53-7, 54-1, 71-2, E-3 Australian, H-1B, H-1B1 Chile, H-1B1 Singapore, AK, AL, AR, AS, AZ, CA,

0.9259054978575781

In [44]:
test_prediction = np.array(predict_test.select('prediction').collect())
train_prediction = np.array(predict_train.select('prediction').collect())
test_label = np.array(predict_test.select('CASE_STATUS').collect())
train_label = np.array(predict_train.select('CASE_STATUS').collect())


21/10/31 01:28:45 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: AK, AL, AR, AZ, CA, CO, CT, DC, DE, FL, FM, GA, GU, HI, IA, ID, IL, IN, KS, KY, LA, MA, MD, ME, MH, MI, MN, MO, MP, MS, MT, NC, ND, NE, NH, NJ, NM, NV, NY, OH, OK, OR, PA, PR, PW, RI, SC, SD, TN, TX, UT, VA, VI, VT, WA, WI, WV, WY, 11-1, 11-2, 11-3, 11-9, 13-1, 13-2, 15-1, 15-2, 15-9, 16-1, 17-1, 17-2, 17-3, 19-1, 19-2, 19-3, 19-4, 20-2, 21-1, 21-2, 23-1, 23-2, 25-1, 25-2, 25-3, 25-4, 25-7, 25-9, 26-1, 27-1, 27-2, 27-3, 27-4, 29-1, 29-2, 29-9, 31-1, 31-2, 31-9, 33-1, 33-2, 33-3, 33-9, 35-1, 35-2, 35-3, 35-9, 37-1, 37-2, 37-3, 39-1, 39-2, 39-3, 39-4, 39-5, 39-6, 39-7, 39-9, 40-9, 41-1, 41-2, 41-3, 41-4, 41-9, 43-1, 43-3, 43-4, 43-5, 43-6, 43-9, 45-1, 45-2, 45-3, 45-4, 47-1, 47-2, 47-3, 47-4, 47-5, 49-1, 49-2, 49-3, 49-9, 51-1, 51-2, 51-3, 51-4, 51-5, 51-6, 51-7, 51-8, 51-9, 53-1, 53-2, 53-3, 53-5, 53-6, 53-7, 54-1, 71-2, E-3 Australian, H-1B, H-1B1 Chile, H-1B1 Singapore, AK, AL, AR, AS, AZ, CA,

In [45]:
from sklearn.metrics import f1_score, balanced_accuracy_score, accuracy_score
print(f1_score(test_label, test_prediction, average='macro'),
      f1_score(test_label, test_prediction, average='micro'))
print(balanced_accuracy_score(test_label, test_prediction),
      accuracy_score(test_label, test_prediction))


0.6074540358900252 0.9377970454246791
0.5437333577286547 0.9377970454246791


In [46]:
print(f1_score(train_label, train_prediction, average='macro'),
      f1_score(train_label, train_prediction, average='micro'))
print(balanced_accuracy_score(train_label, train_prediction),
      accuracy_score(train_label, train_prediction))


0.6077890598228812 0.9375050232639454
0.5432141215991315 0.9375050232639454


In [47]:
f1_score(train_label, train_prediction, average=None)

array([0.39663754, 0.96793573, 0.81959982, 0.24698315])

21/11/01 04:54:36 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 278241 ms exceeds timeout 120000 ms
21/11/01 04:54:36 WARN SparkContext: Killing executors is not supported by current scheduler.
